In [1]:
import pandas as pd
import numpy as np
!pip install ortools
from ortools.linear_solver import pywraplp

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


## Fantasy Football Linear Program 
Emily Daskas

##### Parameters
- max budget of 200
- roster has exactly 15 players
- roster has 1 QB, 3 WR, 2 RB, 1 TE, 1 K, 1 DEF, 6 Bench
- a slot can only be taken up by a player of that type
- a Bench slot can be ay player type
* a player on your roster can only be assigned to one slot
* at most 2 QB and 2 TE on your roster
* at most 1K and 1DEF on your roster
* a player on bench will only generate 1/2^d of their project points 
    * d is the depth of their position
        * example: you havee RB1, RB2, RB3, RB4 on your team with projected points P1, P2, P3, P4
        * the expacted points from these player would be $P1 + P2 + 1/2 * P2 + 1/4 * P3$

##### Questions to Answer
- Who would you choose to maximize points while staying within budget?
- How many points is your team expected to generate?
- How much of your budget did you spend?

In [2]:
fantasy_football = pd.read_csv("fantasy_football_data.csv")

## Player Class

In [3]:
class Player:
    
    def __init__(self, name, position, team, projected_points, price):
        self.name = name
        self.position = position
        self.team = team
        self.projected_points = projected_points
        self.price = price
        
players_list = []

for idx, fantasy_football in fantasy_football.iterrows():
    players_list.append(Player(fantasy_football['Name'], fantasy_football['Position'], fantasy_football['Team'], fantasy_football['ProjectedPoints'], fantasy_football['Price']))

## Functions for LP 

In [4]:
def set_price_constraints(players, players_list):
    for p in players:
        priceConstraint.SetCoefficient(p, players_list[i].price)  
        
def set_coefficients(players):
    for p in players:
        player.SetCoefficient(p, 1)
        
def set_roster_constraints(players):
    for p in players:
        roster_constraint.SetCoefficient(p, 1)  
        
def add_solvers(players):
    for p in players:
        solver_vars.append(p)
        
def set_position_bench_constraints(players, bench_constraints):
    bench_players = players[1:]
    for i in range(len(bench_players)):
        bench_constraints[i].SetCoefficient(bench_players[i], 1)
        
def set_player_bench_constraints(players):
    bench_players = players[1:]
    for player in bench_players:
        bench_constraint.SetCoefficient(player, 1)
        
def set_total_player(players, constraint):
    for player in players:
        constraint.SetCoefficient(player,1)

        
def set_objective_coefficient(players, players_list, idx):
    position = 0
    for player in players:
        objective.SetCoefficient(player, players_list[idx].projected_points * (.5**position))
        position += 1

          
def set_player_position_limits(num_spots):
    spot = ""
    position_array = []
    
    for j in range(num_spots):
        if j == 0:
            spot = '(active)'
        else:
            spot = '(bench)'
            
        position_array.append(solver.IntVar(0,1, players_list[i].name + ': {}{} {}'.format(players_list[i].position, j+1, spot)))
    
    return position_array

## Linear Program

In [5]:
solver = pywraplp.Solver.CreateSolver('SCIP')

# constraint vars

budget = 200

roster_length = 15

max_player_occurance = 1

benched_players = 6

roster_constraints = {
    'active_QB_max': 1, 'QB_total_max': 2,
    'active_WR_max': 3,
    'active_RB_max': 2,
    'active_K_max': 1,
    'active_DEF_max': 1, 'DEF_total_max': 1,
    'active_TE_max': 1, 'TE_total_max': 2,
}

In [6]:
# setting constraints

priceConstraint = solver.Constraint(0, budget, "max_budget")

roster_constraint = solver.Constraint(0, roster_length, "max_roster")
bench_constraint = solver.Constraint(0, benched_players , "max_bench_size")

active_QB_constraint = solver.Constraint(0, roster_constraints['active_QB_max'], "max_active_QB")
active_RB_constraint = solver.Constraint(0, roster_constraints['active_RB_max'], "max_active_RB")
active_WR_constraint = solver.Constraint(0, roster_constraints['active_WR_max'], "max_active_WR")
active_TE_constraint = solver.Constraint(0, roster_constraints['active_TE_max'], "max_active_TE")
active_K_constraint = solver.Constraint(0, roster_constraints['active_K_max'], "max_active_K")
active_DEF_constraint = solver.Constraint(0, roster_constraints['active_DEF_max'], "max_active_DEF")

total_QB_constraint = solver.Constraint(0, roster_constraints['QB_total_max'], "max_total_QB")
total_TE_constraint = solver.Constraint(0, roster_constraints['TE_total_max'], "max_total_TE")
total_K_constraint  = solver.Constraint(0, roster_constraints['active_K_max'], "max_total_K")
total_DEF_constraint = solver.Constraint(0, roster_constraints['DEF_total_max'], "max_total_DEF")


running_back_bench_constraints = []
for i in range(benched_players):
    running_back_bench_constraints.append(solver.Constraint(0, max_player_occurance, "running_back_bench_{}".format(i)))

    
wide_receiver_bench_constraints = []
for i in range(benched_players):
    wide_receiver_bench_constraints.append(solver.Constraint(0, max_player_occurance, "wide_receiver_bench_{}".format(i)))


tight_end_bench_constraints = []
for i in range(2):
    tight_end_bench_constraints.append(solver.Constraint(0, max_player_occurance, "tight_end_bench_{}".format(i)))


objective = solver.Objective()
objective.SetMaximization()

solver_vars = []

#lp
for i in range(len(players_list)):
    
    player = solver.Constraint(0, 1, players_list[i].name)
    
    if players_list[i].position == 'QB':
        
        qbs = set_player_position_limits(2) #bench + active
        
        add_solvers(qbs) 
        
        set_coefficients(qbs)
    
        set_price_constraints(qbs, players_list)

        set_roster_constraints(qbs)

        set_total_player(qbs, total_QB_constraint)

        active_QB_constraint.SetCoefficient(qbs[0], 1)
    
        bench_constraint.SetCoefficient(qbs[1], 1)
            
        objective.SetCoefficient(qbs[0], players_list[i].projected_points)
    
        objective.SetCoefficient(qbs[1], players_list[i].projected_points * .5)

    
    elif players_list[i].position == 'RB':

        rbs = set_player_position_limits(len(running_back_bench_constraints) + 1)
        
        add_solvers(rbs) 

        set_coefficients(rbs)

        set_price_constraints(rbs, players_list)

        set_roster_constraints(rbs)
 
        active_RB_constraint.SetCoefficient(rbs[0], 1)
            
        set_position_bench_constraints(rbs, running_back_bench_constraints)

        set_player_bench_constraints(rbs)
        
        set_objective_coefficient(rbs, players_list, i)


    elif players_list[i].position == 'WR':
        
        wrs = set_player_position_limits(len(wide_receiver_bench_constraints) + 1) 

        add_solvers(wrs)

        set_coefficients(wrs)    

        set_price_constraints(wrs, players_list)

        set_roster_constraints(wrs)
        
        active_WR_constraint.SetCoefficient(wrs[0], 1)
        
        set_position_bench_constraints(wrs, wide_receiver_bench_constraints)

        set_player_bench_constraints(wrs)

        set_objective_coefficient(wrs, players_list, i)


    elif players_list[i].position == 'K':
        
        ks = [solver.IntVar(0, 1, players_list[i].name + ': K (active)')]
        
        add_solvers(ks)
        
        set_coefficients(ks)
        
        set_price_constraints(ks, players_list)

        set_roster_constraints(ks)

        total_K_constraint.SetCoefficient(ks[0], 1)
        
        active_K_constraint.SetCoefficient(ks[0], 1)
        
        set_objective_coefficient(ks, players_list, i)        
  

    elif players_list[i].position == 'TE':
        
        tes = set_player_position_limits(len(tight_end_bench_constraints) + 1)
                
        add_solvers(tes)

        set_coefficients(tes)

        set_price_constraints(tes, players_list)

        set_roster_constraints(tes)

        set_total_player(tes, total_TE_constraint)

        active_TE_constraint.SetCoefficient(tes[0], 1)
        
        set_position_bench_constraints(tes, tight_end_bench_constraints)

        set_player_bench_constraints(tes)

        set_objective_coefficient(tes, players_list, i)


    elif players_list[i].position == 'DEF':
        
        defs = [solver.IntVar(0, 1, players_list[i].name + ' DEF')]
        
        add_solvers(defs)
        
        set_coefficients(defs)

        set_price_constraints(defs, players_list)

        set_roster_constraints(defs)

        total_DEF_constraint.SetCoefficient(defs[0], 1)

        active_DEF_constraint.SetCoefficient(defs[0], 1)
    
        set_objective_coefficient(defs, players_list, i)
      
        
status = solver.Solve()      
total_cost = 0


## Results

In [7]:
print("Fantasy Football Roster:\n")
if status== pywraplp.Solver.OPTIMAL:
    total_cost = 0
    
    for roster_player in solver_vars:
        
        if roster_player.solution_value() == 1:
            print(roster_player.name())
            total_cost += priceConstraint.GetCoefficient(roster_player)


print()
print("My team is expected to generate", solver.Objective().Value(), "points.")
print()
print("100% of the budget was used: ${}".format(total_cost))
    

Fantasy Football Roster:

Cam Newton: QB1 (active)
Tyrod Taylor: QB2 (bench)
Ty Montgomery: RB1 (active)
C.J. Anderson: RB1 (active)
Jonathan Stewart: RB2 (bench)
Theo Riddick: RB3 (bench)
Antonio Brown: WR1 (active)
Julio Jones: WR1 (active)
Emmanuel Sanders: WR1 (active)
Rishard Matthews: WR2 (bench)
John Brown: WR3 (bench)
Justin Tucker: K (active)
Martellus Bennett: TE1 (active)
Coby Fleener: TE2 (bench)
Denver Broncos DEF

My team is expected to generate 1754.25 points.

100% of the budget was used: $200.0
